# HW-aware Synthesis of MCX

This example shows that implementation of multiple control-x (MCX) logic, using the Classiq synthesis engine, yields different circuit results for different quantum hardware.

The fictitious hardware created here demonstrates how to insert your own custom-designed machine. For comparison, create two types of hardware with `cx, u` basis gates. The difference between them manifests in the connectivity map: one has linear connectivity while the other has all-to-all connectivity.

In [4]:
from classiq import (
    CX,
    GeneratedCircuit,
    H,
    Output,
    QArray,
    QBit,
    QFunc,
    X,
    allocate,
    control,
    create_model,
    set_constraints,
    set_preferences,
    show,
    synthesize,
)
from classiq.model import (
    Constraints,
    CustomHardwareSettings,
    OptimizationParameter,
    Preferences,
)

# define MCX quantum function


@QFunc
def my_mcx(cntrl: QArray[QBit], target: QBit) -> None:
    control(lambda: X(target), cntrl)


# define the MCX parameters within the quantum 'main' function
@QFunc
def main(cntrl: Output[QArray[QBit]], target: Output[QBit]) -> None:
    allocate(15, cntrl)
    allocate(1, target)
    my_mcx(cntrl, target)


# build a model
model = create_model(main)

# define the hardware parameters
max_width = 18

linear_connectivity = [[qubit, qubit + 1] for qubit in range(max_width - 1)]

preferences_linear = Preferences(
    custom_hardware_settings=CustomHardwareSettings(
        basis_gates=["cx", "u"],
        connectivity_map=linear_connectivity,
    ),
    random_seed=-1,
)
preferences_all_to_all = Preferences(
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    random_seed=-1,
)

# define synthesis engine constraints

constraints = Constraints(optimization_parameter="depth", max_width=max_width)

# define models with different preferences
model = set_constraints(model, constraints)

model_linear = set_preferences(model, preferences_linear)
model_all_to_all = set_preferences(model, preferences_all_to_all)

# write models to files

with open("hardware_aware_mcx_linear.qmod", "w") as f:
    f.write(model_linear)

with open("hardware_aware_mcx_all_to_all.qmod", "w") as f:
    f.write(model_all_to_all)


# synthesize to create quantum programs and view circuits:
qprog_linear = synthesize(model_linear)
show(qprog_linear)

qprog_all_to_all = synthesize(model_all_to_all)
show(qprog_all_to_all)

Comparison of the two circuits shows that applying MCx using different connectivity maps yields different implementation.

Using "all-to-all" connectivity, the synthesis engine chooses as the best implementation a recourse based on "Maslov2015" [[1](#Maslov)] that was written on the Classiq platform. Using that, the manufactured circuit has 18 qubits; i.e., it uses two auxiliary qubits. The total depth of the circuit is 936.

When using linear connectivity, the best implementation chosen by the synthesis engine is, in fact, different: an algorithm developed by Classiq, which is better suited for this map. Here, the manufactured circuit uses 17 qubits with only one auxiliary and has a depth of 1257 gates.

## References

<a id='Maslov'>[1]</a>: [Maslov, D., 2016. Advantages of using relative-phase Toffoli gates with an application to multiple control Toffoli optimization. Physical Review A, 93(2), p.022311.](https://arxiv.org/pdf/1508.03273.pdf)